In [1]:
import numpy as np
import pandas as pd
import open3d as o3d

from pathlib import Path

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import utils
import calibration_utils

import importlib
importlib.reload(utils)
importlib.reload(calibration_utils)

<module 'calibration_utils' from '/home/tom/Documents/UNT/csce6260/projects/kitti-experiments/fusion/calibration_utils.py'>

In [3]:
base = Path.home() / "kitti"
train_dir = base / "training"
train_labels_dir = train_dir / "label_2"
kitti_train_labels = sorted(train_labels_dir.glob("*.txt"))
velo_dir = train_dir / "velodyne"
point_cloud_train_files = sorted(velo_dir.glob("*.bin"))
calib_dir = train_dir / "calib"
calib_train_files = sorted(calib_dir.glob("*.txt"))

In [7]:
def get_point_cloud(points_velo):
    cloud = o3d.geometry.PointCloud()
    cloud.points = o3d.utility.Vector3dVector(points_velo[:, :3])
    return cloud

def downsample(cloud):
    cloud = cloud.voxel_down_sample(voxel_size=0.2)
    return cloud

def segment(cloud):
    plane_model, inliers = cloud.segment_plane(distance_threshold=0.3, ransac_n=3, num_iterations=150)
    outlier_cloud = cloud.select_by_index(inliers, invert=True)
    return outlier_cloud

def get_downsampled_point_cloud(points_velo):
    cloud = get_point_cloud(points_velo)
    cloud = downsample(cloud)
    cloud = segment(cloud)
    return cloud

def depth_filter(cloud):
    cloud_points = np.array(cloud.points)
    mask_in_front = cloud_points[:, 0] > 0
    cloud_points_in_cam_front = cloud_points[mask_in_front]
    return cloud_points_in_cam_front

def get_points_uv(cloud_points_in_cam_front, calib):
    points_uv = calibration_utils.convert_to_img_coords(cloud_points_in_cam_front, calib)
    return points_uv[:, :2]

def get_labels(label_file_path):
    labels = utils.parse_label_file(label_file_path)
    labels = [label for label in labels if label['type'] in ['Car', 'Pedestrian', 'Cyclist']]
    return labels

def get_mask_frustums(points_uv, labels):
    mask_frustums = []
    u, v = points_uv[:, 0], points_uv[:, 1]
    valid_mask = np.isfinite(u) & np.isfinite(v)
    for label in labels:
        xmin, ymin, xmax, ymax = label['bbox_2d']
        mask_frustum = (u >= xmin) & (u <= xmax) & (v >= ymin) & (v <= ymax)
        mask_frustum = mask_frustum & valid_mask
        mask_frustums.append(mask_frustum)
    return mask_frustums

def extract_frustum_points(mask_frustums, cloud_points_in_cam_front):
    frustum_points = []
    for mask_frustum in mask_frustums:
        points = cloud_points_in_cam_front[mask_frustum]
        frustum_points.append(points)
    return frustum_points

def get_frustum(points):
    frustum = o3d.geometry.PointCloud()
    frustum.points = o3d.utility.Vector3dVector(points[:, :3])
    return frustum

def get_cluster_labels(frustum):
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
        cluster_labels = np.array(frustum.cluster_dbscan(eps=0.45, min_points=10, print_progress=True))
    return cluster_labels

def group_cluster_labels(cluster_labels):
    valid_mask = cluster_labels != -1
    return (
        pd.Series(np.arange(len(cluster_labels)))[valid_mask]
        .groupby(cluster_labels[valid_mask], sort=False)
        .apply(list)
        .tolist()
    )

def get_frustum_with_clusters(points):
    frustum = get_frustum(points)
    cluster_labels = get_cluster_labels(frustum)
    return frustum, group_cluster_labels(cluster_labels)    

def get_frustums_with_clusters(frustum_points):
    frustums_with_clusters = []
    for i, points in enumerate(frustum_points):
        frustum, clusters = get_frustum_with_clusters(points)
        frustums_with_clusters.append((frustum, clusters,))
    return frustums_with_clusters

for analysis_file_index in range(100):
    bin_path = point_cloud_train_files[analysis_file_index]
    points_velo = utils.read_velodyne_bin(bin_path)
    cloud = get_downsampled_point_cloud(points_velo)
    cloud_points_in_cam_front = depth_filter(cloud)
    calib = utils.parse_calib_file(calib_train_files[analysis_file_index])
    points_uv = get_points_uv(cloud_points_in_cam_front, calib)
    labels = get_labels(kitti_train_labels[analysis_file_index])
    mask_frustums = get_mask_frustums(points_uv, labels)
    frustum_points = extract_frustum_points(mask_frustums, cloud_points_in_cam_front)
    frustums_with_clusters = get_frustums_with_clusters(frustum_points)
    break

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 5
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 2
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 3
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 1
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 1
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 3
[38, 10, 12, 10, 10]
[43, 34]
[141, 14, 11]
Precompute neighbors.[============================